In [1]:
import pandas as pd
import numpy as np

from scipy import stats
import statsmodels.stats.multitest as smm

In [2]:
count_m = pd.read_csv('gene_high_throughput_sequencing.csv')

count_m.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [3]:
count_m.Diagnosis.value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

In [4]:
normal = count_m.loc[count_m.Diagnosis == 'normal']

cancer = count_m.loc[count_m.Diagnosis == 'cancer']

early_n = count_m.loc[count_m.Diagnosis == 'early neoplasia']

In [5]:
stat, p = stats.ttest_ind(normal.iloc[:, 2], early_n.iloc[:, 2])
stat, p

(0.40050760239317207, 0.6905974705672382)

In [6]:
normal['LOC643837']

0     1.257614
1     4.567931
2     2.077597
3     2.066576
4     2.613616
5     3.942275
6     1.084113
7     3.153900
8     2.551800
9     3.693128
10    3.558222
11    0.938061
12    1.003451
13    7.364879
14    2.561871
15    2.971205
16    2.871767
17    1.045382
18    1.801865
19    3.515834
20    2.234576
21    4.717822
22    1.474174
23    1.282995
Name: LOC643837, dtype: float64

In [7]:
normal_vs_early = []

for i, d in (zip(normal.columns[2:], early_n.columns[2:])):
    
    mean1, mean2 = np.mean(normal[i]), np.mean(early_n[d])
    
    if mean2 > mean1:
        lfc = mean2 / mean1
    else:
        lfc = - mean1 / mean2
        
    stat, p = stats.ttest_ind(normal[i], early_n[d], equal_var=False)
    normal_vs_early.append([i, d, mean1, mean2, lfc, stat, p])
            

In [8]:
normal_vs_early = pd.DataFrame.from_records(normal_vs_early)
normal_vs_early.columns = ['norm_g', 'early_g', 'norm_mean', 'early_mean', 'lfc','stat', 'p']
normal_vs_early

,norm_g,early_g,norm_mean,early_mean,lfc,stat,p
0,LOC643837,LOC643837,2.681277,2.510894,-1.067858,0.400289,0.690766
1,LOC100130417,LOC100130417,4.368497,8.721781,1.996517,-4.608766,0.000032
2,SAMD11,SAMD11,15.159566,18.531325,1.222418,-1.929277,0.060273
3,NOC2L,NOC2L,15.374351,15.071854,-1.020070,0.220542,0.826429
4,KLHL17,KLHL17,21.459886,24.152469,1.125471,-2.013201,0.049876
...,...,...,...,...,...,...,...
15743,DDX3Y,DDX3Y,1.701654,1.961449,1.152672,-0.470046,0.640904
15744,CD24,CD24,17.106405,17.567902,1.026978,-0.215891,0.830134
15745,CYorf15B,CYorf15B,1.576004,1.760707,1.117197,-0.428747,0.670395
15746,KDM5D,KDM5D,1.960442,2.151549,1.097481,-0.262910,0.793925


In [9]:
early_vs_cancer = []

for i, d in (zip(early_n.columns[2:], cancer.columns[2:])):
    mean1, mean2 = np.mean(early_n[i]), np.mean(cancer[d])
    
    if mean2 > mean1:
        lfc = mean2 / mean1
    else:
        lfc = - mean1 / mean2
        
    stat, p = stats.ttest_ind(early_n[i], cancer[d], equal_var=False)
    early_vs_cancer.append([i, d, mean1, mean2, lfc, stat, p])

In [10]:
early_vs_cancer = pd.DataFrame.from_records(early_vs_cancer)
early_vs_cancer.columns = ['early_g', 'cancer_g', 'early_mean', 'cancer_mean', 'lfc', 'stat', 'p']
early_vs_cancer

,early_g,cancer_g,early_mean,cancer_mean,lfc,stat,p
0,LOC643837,LOC643837,2.510894,2.186060,-1.148593,0.824849,0.413735
1,LOC100130417,LOC100130417,8.721781,8.190456,-1.064871,0.452236,0.653429
2,SAMD11,SAMD11,18.531325,23.692614,1.278517,-1.817266,0.079556
3,NOC2L,NOC2L,15.071854,16.468034,1.092635,-1.075985,0.287581
4,KLHL17,KLHL17,24.152469,25.035813,1.036574,-0.740329,0.463292
...,...,...,...,...,...,...,...
15743,DDX3Y,DDX3Y,1.961449,2.292908,1.168987,-0.443835,0.659369
15744,CD24,CD24,17.567902,15.752268,-1.115262,0.983573,0.330617
15745,CYorf15B,CYorf15B,1.760707,2.157294,1.225243,-0.613696,0.542939
15746,KDM5D,KDM5D,2.151549,2.776173,1.290314,-0.579209,0.565753


In [11]:
print((early_vs_cancer.p < 0.05).sum())
print((normal_vs_early.p < 0.05).sum())

3490
1575


In [12]:
len(early_vs_cancer.p < 0.05)

15748

In [13]:
early_vs_cancer[early_vs_cancer.p < 0.05]

,early_g,cancer_g,early_mean,cancer_mean,lfc,stat,p
5,PLEKHN1,PLEKHN1,10.793160,13.870940,1.285160,-2.803258,0.007681
8,ISG15,ISG15,37.296084,91.706112,2.458867,-3.904478,0.000740
12,TTLL10,TTLL10,7.832093,5.917256,-1.323602,2.845402,0.006844
13,TNFRSF18,TNFRSF18,22.943534,30.130816,1.313260,-3.268558,0.002094
16,B3GALT6,B3GALT6,16.505549,20.071459,1.216043,-2.237967,0.031126
...,...,...,...,...,...,...,...
15722,FAM50A,FAM50A,54.403960,60.715148,1.116006,-2.124856,0.039011
15723,PLXNA3,PLXNA3,27.982613,34.439680,1.230753,-2.954959,0.005073
15724,LAGE3,LAGE3,25.205297,35.950025,1.426289,-3.034358,0.004185
15726,SLC10A3,SLC10A3,10.092209,11.819726,1.171173,-2.239218,0.030746


In [14]:
normal_vs_early.loc[normal_vs_early.p < 0.05]

,norm_g,early_g,norm_mean,early_mean,lfc,stat,p
1,LOC100130417,LOC100130417,4.368497,8.721781,1.996517,-4.608766,0.000032
4,KLHL17,KLHL17,21.459886,24.152469,1.125471,-2.013201,0.049876
7,HES4,HES4,58.080871,64.670644,1.113459,-2.337789,0.023712
9,AGRN,AGRN,62.870812,75.707467,1.204175,-2.138607,0.037933
10,C1orf159,C1orf159,40.998143,47.200942,1.151295,-2.412757,0.020002
...,...,...,...,...,...,...,...
15687,ZNF185,ZNF185,10.998709,12.612792,1.146752,-2.083196,0.043227
15700,SRPK3,SRPK3,9.904238,7.679047,-1.289774,2.151725,0.036647
15726,SLC10A3,SLC10A3,12.109389,10.092209,-1.199875,2.801084,0.007540
15728,G6PD,G6PD,47.952482,59.399255,1.238711,-2.224274,0.032166


In [15]:
normal.info(), cancer.info(), early_n.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 23
Columns: 15750 entries, Patient_id to EIF1AY
dtypes: float64(15748), object(2)
memory usage: 2.9+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 49 to 71
Columns: 15750 entries, Patient_id to EIF1AY
dtypes: float64(15748), object(2)
memory usage: 2.8+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 24 to 48
Columns: 15750 entries, Patient_id to EIF1AY
dtypes: float64(15748), object(2)
memory usage: 3.0+ MB


(None, None, None)

In [16]:
def write_ans(name, data):
    with open(name, 'w') as f:
        f.write(str(data))
        
write_ans('ans1_gene.txt', 1575)
write_ans('ans2_gene.txt', 3490)

In [17]:
reject, p_corrected, a1, a2 = smm.multipletests(normal_vs_early.p, 
                                            alpha = 0.025, 
                                            method = 'holm')
normal_vs_early['p_corrected'] = p_corrected
normal_vs_early['reject'] = reject

In [18]:
normal_vs_early.loc[normal_vs_early['reject'] == True]

,norm_g,early_g,norm_mean,early_mean,lfc,stat,p,p_corrected,reject
7244,PCSK4,PCSK4,14.983424,22.621756,1.509785,-5.806801,7.955435e-07,0.012527,True
9820,EEF1A2,EEF1A2,19.373612,38.260320,1.974868,-6.524922,8.498742e-08,0.001338,True


In [19]:
write_ans('ans3_gene.txt', 2)

In [20]:
reject, p_corrected, a1, a2 = smm.multipletests(early_vs_cancer.p, 
                                            alpha = 0.025, 
                                            method = 'holm')
early_vs_cancer['p_corrected'] = p_corrected
early_vs_cancer['reject'] = reject

In [21]:
early_vs_cancer.loc[early_vs_cancer['reject'] == True] 

,early_g,cancer_g,early_mean,cancer_mean,lfc,stat,p,p_corrected,reject
47,GABRD,GABRD,7.224275,13.320553,1.843860,-6.583440,2.276043e-07,0.003575,True
283,EXTL1,EXTL1,23.234551,9.749268,-2.383210,6.600818,1.087046e-07,0.001709,True
316,CD164L2,CD164L2,23.734858,11.882991,-1.997381,5.826854,1.002085e-06,0.015708,True
1105,NES,NES,93.810978,55.397100,-1.693428,6.709055,1.041679e-07,0.001637,True
1238,FMO2,FMO2,11.619290,6.470836,-1.795640,5.751625,1.103311e-06,0.017292,True
...,...,...,...,...,...,...,...,...,...
15090,LAMC3,LAMC3,40.668287,14.175370,-2.868940,8.424379,1.938708e-09,0.000031,True
15449,SLC7A3,SLC7A3,9.553186,2.962692,-3.224495,7.046264,3.981097e-08,0.000626,True
15576,CAPN6,CAPN6,10.070903,3.935252,-2.559150,6.524854,7.554205e-08,0.001188,True
15640,GPC3,GPC3,10.280356,5.496242,-1.870434,5.805582,5.832350e-07,0.009152,True


In [22]:
early_vs_cancer[(early_vs_cancer['reject'] == True) & (abs(early_vs_cancer['lfc']) > 1.5)]

,early_g,cancer_g,early_mean,cancer_mean,lfc,stat,p,p_corrected,reject
47,GABRD,GABRD,7.224275,13.320553,1.843860,-6.583440,2.276043e-07,0.003575,True
283,EXTL1,EXTL1,23.234551,9.749268,-2.383210,6.600818,1.087046e-07,0.001709,True
316,CD164L2,CD164L2,23.734858,11.882991,-1.997381,5.826854,1.002085e-06,0.015708,True
1105,NES,NES,93.810978,55.397100,-1.693428,6.709055,1.041679e-07,0.001637,True
1238,FMO2,FMO2,11.619290,6.470836,-1.795640,5.751625,1.103311e-06,0.017292,True
...,...,...,...,...,...,...,...,...,...
15090,LAMC3,LAMC3,40.668287,14.175370,-2.868940,8.424379,1.938708e-09,0.000031,True
15449,SLC7A3,SLC7A3,9.553186,2.962692,-3.224495,7.046264,3.981097e-08,0.000626,True
15576,CAPN6,CAPN6,10.070903,3.935252,-2.559150,6.524854,7.554205e-08,0.001188,True
15640,GPC3,GPC3,10.280356,5.496242,-1.870434,5.805582,5.832350e-07,0.009152,True


In [23]:
write_ans('ans4_gene.txt', 77)

In [24]:
reject, p_corrected, a1, a2 = smm.multipletests(normal_vs_early.p, 
                                            alpha = 0.025, 
                                            method = 'fdr_bh')
normal_vs_early['p_corrected'] = p_corrected
normal_vs_early['reject'] = reject

len(normal_vs_early[(normal_vs_early['reject'] == True) & (abs(normal_vs_early['lfc']) > 1.5)])


4

In [25]:
write_ans('ans5_gene.txt', 4)

In [26]:
reject, p_corrected, a1, a2 = smm.multipletests(early_vs_cancer.p, 
                                            alpha = 0.025, 
                                            method = 'fdr_bh')
early_vs_cancer['p_corrected'] = p_corrected
early_vs_cancer['reject'] = reject

len(early_vs_cancer[(early_vs_cancer['reject'] == True) & (abs(early_vs_cancer['lfc']) > 1.5)])


524

In [27]:
write_ans('ans6_gene.txt', 524)